In [1]:
import json
import nltk
import string
import numpy as np
import pandas as pd
from unidecode import unidecode
import transformers
import torch
from sklearn.metrics.pairwise import cosine_similarity
from pathlib import Path
from tqdm import tqdm

from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import RetrievalQA
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

/home/super-pc-linux/miniconda3/envs/biorag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setup device

In [2]:
device = (
  'cuda'
  if torch.cuda.is_available()
  else 'mps'
  if torch.backends.mps.is_available()
  else 'cpu'
)

## Setup metric

In [3]:
lemmatizer = nltk.stem.WordNetLemmatizer()

def preprocess(corpus):
  corpus = corpus.lower()
  stopset = nltk.corpus.stopwords.words('english') + nltk.corpus.stopwords.words('russian') + list(string.punctuation)
  tokens = nltk.word_tokenize(corpus)
  tokens = [t for t in tokens if t not in stopset]
  tokens = [lemmatizer.lemmatize(t) for t in tokens]
  corpus = ' '.join(tokens)
  corpus = unidecode(corpus)
  return corpus

In [4]:
embeddings = OllamaEmbeddings(model='llama3')

In [5]:
def embeddings_cosine_sim_metric(expected_answers, predicted_answers):
  results = []

  for expected_answer, predicted_answer in zip(expected_answers, predicted_answers):
    expected_answer = preprocess(expected_answer)
    predicted_answer = preprocess(predicted_answer)

    expected_embedding = np.array(embeddings.embed_query(expected_answer))
    predicted_embedding = np.array(embeddings.embed_query(predicted_answer))

    sim = cosine_similarity(
      expected_embedding.reshape(1, -1),
      predicted_embedding.reshape(1, -1),
    )[0][0]

    results.append(sim)

  return np.mean(results)

## Load QA dataset

In [6]:
qa_df = pd.read_csv('../research-neurobiology-qa-dataset/brainscape.csv')
qa_df

,question,answer
0,What are the afferent cranial nerve nuclei?,Trigeminal sensory nucleus- fibres carry gener...
1,What is the order of the cranial nerves ?,1-olfactory\n2-optic\n3-oculomotor\n4-trochlea...
2,What are the efferent cranial nerve nuclei?,Edinger-westphal nucleus\nOculomotor nucleus\n...
3,Which nuclei share the embryo logical origin -...,Oculomotor nucleus Trochlear nucleus Abducens ...
4,Which nuclei share the embryo logical origin- ...,Trigeminal motor nucleus Facial motor nucleus ...
...,...,...
1047,What is the purpose of gephyrin in the glycine...,Involved in anchoring the receptor to a specif...
1048,What is the glycine receptor involved in ?,Reflex response\nCauses reciprocal inhibition ...
1049,What happens in hyperperplexia ?,It’s an exaggerated reflex Often caused by a m...
1050,What is hyperperplexia treated with ?,Benzodiazepine


## Load documents

In [7]:
docs_dir = Path('./docs')
loaders = []

for file in tqdm(docs_dir.iterdir()):
  if file.is_file() and file.suffix == '.pdf':
    loader = PyPDFLoader(file)
    loaders.append(loader)

6it [00:00, 51.06it/s]


## Setup LLMs

In [8]:
def get_llama2():
  return Ollama(temperature=0, model='llama2')

In [9]:
def get_llama3():
  return Ollama(temperature=0, model='llama3')

In [10]:
def openbiollm_parser(output):
  idx = output.find('Helpful Answer: ')
  if idx != -1:
    return output[idx + len('Helpful answer: '):]
  else:
    return output

def get_openbiollm_8b():
  model = 'aaditya/OpenBioLLM-Llama3-8B'
  model_kwargs = {'torch_dtype': torch.bfloat16}
  pipeline = transformers.pipeline(
    'text-generation',
    model=model,
    model_kwargs=model_kwargs,
    device=device,
  )
  terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids('<|eot_id|>')
  ]
  llm = HuggingFacePipeline.from_model_id(
    model_id=model,
    task='text-generation',
    model_kwargs=model_kwargs,
    pipeline_kwargs={
      'max_new_tokens': 256,
      'eos_token_id': terminators,
      'do_sample': True,
      'temperature': 0.01,
      'top_p': 0.9,
    },
  )
  return llm | openbiollm_parser

In [11]:
def openbiollm_parser(output):
  idx = output.find('Helpful Answer: ')
  if idx != -1:
    return output[idx + len('Helpful answer: '):]
  else:
    return output

def get_miread_neuro():
  model = 'aaditya/OpenBioLLM-Llama3-8B'
  model_kwargs = {'torch_dtype': torch.bfloat16}
  pipeline = transformers.pipeline(
    'text-generation',
    model=model,
    model_kwargs=model_kwargs,
    device=device,
  )
  terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids('<|eot_id|>')
  ]
  llm = HuggingFacePipeline.from_model_id(
    model_id=model,
    task='text-generation',
    model_kwargs=model_kwargs,
    pipeline_kwargs={
      'max_new_tokens': 256,
      'eos_token_id': terminators,
      'do_sample': True,
      'temperature': 0.01,
      'top_p': 0.9,
    },
  )
  return llm | openbiollm_parser

## Setup index stores

In [12]:
def get_doc_array_in_memory_search(loaders=[]):
  index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
  ).from_loaders(loaders)
  return index.vectorstore

## Setup experiments

In [13]:
llms = [
  ('LLaMA 2', get_llama2()),
  ('LLaMA 3', get_llama3()),
  ('OpenBioLLM Llama3 8B', get_openbiollm_8b()),
]

vector_stores = [
  ('Doc Array In Memory Search', get_doc_array_in_memory_search),
]

Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.91s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.19it/s]
Device has 1 GPUs available. Provide device={deviceId} to `from_model_id` to use availableGPUs for execution. deviceId is -1 (default) for CPU and can be a positive integer associated with CUDA device id.


In [14]:
cache_path = Path('cache.json')
with open(cache_path, 'r') as f:
  cache = json.load(f)

cache.keys()

dict_keys(['LLaMA 3_Doc Array In Memory Search_True', 'LLaMA 3_Doc Array In Memory Search_False', 'OpenBioLLM Llama3 8B_Doc Array In Memory Search_True', 'LLaMA 2_Doc Array In Memory Search_False', 'LLaMA 2_Doc Array In Memory Search_True', 'OpenBioLLM Llama3 8B_Doc Array In Memory Search_False'])

In [15]:
df = pd.DataFrame(columns=['llm', 'vector_store', 'use_docs', 'accuracy'])

sample_df = qa_df.sample(frac=1)
questions = sample_df['question'].tolist()
expected_answers = sample_df['answer'].tolist()

for llm_name, llm in tqdm(llms, desc='LLMs'):
  for vector_store_name, get_vector_store in tqdm(vector_stores, desc='Vector Stores', leave=False):
    for use_docs in tqdm((False, True), desc='Use Docs', leave=False):
      vector_store = get_vector_store(loaders if use_docs else [])
      qa_llm = None

      predicted_answers = []

      for index, question in tqdm(enumerate(questions), desc='Questions', leave=False):
        key = f'{llm_name}_{vector_store_name}_{use_docs}'

        if not key in cache:
          cache[key] = {}

        if not question in cache[key]:
          print(f'{question} is not in {key}')

          if qa_llm == None:
            qa_llm = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type='stuff',
            retriever=vector_store.as_retriever(),
            verbose=False,
            chain_type_kwargs = {'document_separator': '<<<<>>>>>'},
          )

          cache[key][question] = qa_llm.invoke(question)['result']

        predicted_answers.append(cache[key][question])

        with open(cache_path, 'w') as f:
          json.dump(cache, f)

      accuracy = embeddings_cosine_sim_metric(expected_answers, predicted_answers)

      row = pd.DataFrame({
        'llm': llm_name,
        'vector_store': vector_store_name,
        'use_docs': use_docs,
        'accuracy': accuracy,
      }, index=[0])
      df = pd.concat([df, row], ignore_index=True)

LLMs:   0%|          | 0/3 [00:00<?, ?it/s]
/home/super-pc-linux/miniconda3/envs/biorag/lib/python3.12/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


















































































































































































































/tmp/ipykernel_908602/454047965.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, row], ignore_index=True)

Ignoring wrong pointing object 0 0 (offset 0)















































What are neurofibrillary tangles in ad ? is not in OpenBioLLM Llama3 8B_Doc Array In Memory Search_False


Give example of an imidazoleamine is not in OpenBioLLM Llama3 8B_Doc Array In Memory Search_False


Are glutamate and aspartate essential or non essential amino acids ? is not in OpenBioLLM Llama3 8B_Doc Array In Memory Search_False


What are BRAINSTEM gliomas ? is not in OpenBioLLM Llama3 8B_Doc Array In Memory Search_False


...

what cells of the cerebellar cortex are excitatory ? is not in OpenBioLLM Llama3 8B_Doc Array In Memory Search_True


What does the hindbrain include ? is not in OpenBioLLM Llama3 8B_Doc Array In Memory Search_True


What are the 3 main forces that affect blood flow through the lungs ? is not in OpenBioLLM Llama3 8B_Doc Array In Memory Search_True





LLMs:  67%|██████▋   | 2/3 [138:56:57<69:28:28, 250108.57s/it]


KeyboardInterrupt: 

In [ ]:
df

,llm,vector_store,use_docs,accuracy
0,LLaMA 3,Doc Array In Memory Search,True,0.507332
1,LLaMA 3,Doc Array In Memory Search,False,0.444137
